In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
BASE_DIR = "/content/drive/MyDrive/LoCA/"

Mounted at /content/drive


In [2]:
!git clone https://github.com/Bilgecelik/LoCA.git

Cloning into 'LoCA'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 85 (delta 34), reused 61 (delta 19), pack-reused 0
Receiving objects: 100% (85/85), 30.63 KiB | 1.46 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [3]:
!pip install -r "LoCA/requirements"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 894.6/894.6 kB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 93.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.4/532.4 kB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.8/452.8 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 68.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of avalanche-lib[detection] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4

In [ ]:
# CL Setup
import torch
import argparse
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training.supervised import Naive
from avalanche.training.supervised.l2p import LearningToPrompt
from avalanche.logging import WandBLogger, InteractiveLogger
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics
from avalanche.training.plugins import EvaluationPlugin
import wandb
from typing import Dict
WANDB_API_KEY='71b542c3072e07c51d1184841ffc50858ab2090e'


device = torch.device("cuda:0" if device == "gpu" else "cpu")
print(device)
benchmark = PermutedMNIST(n_experiences=number_of_experiences, seed=1)

# set criteria, optimizer (config hp's), model (checkpoint)
criterion = CrossEntropyLoss()
  strategy = LearningToPrompt(
      model_name='vit_large_patch16_224',
      criterion=CrossEntropyLoss(),
      train_mb_size=train_mb_size,
      train_epochs=train_epochs,
      eval_mb_size=eval_mb_size,
      device=device,
      evaluator=eval_plugin,
      num_classes=num_classes,  # total # of classes in all tasks
      use_vit=True,
      lr=learning_rate,
      pool_size=prompt_pool_size,
      prompt_length=prompt_length,
      top_k=top_k,
      sim_coefficient=0.5,  # default in avalanche is 0.1, paper is 0.5, not sensitive
  )

# plot with wandb
wandb_logger = WandBLogger(project_name="LOCA Trials",
                            run_name="CL_Solo",
                            log_artifacts=True,
                            config={
                                "dataset": benchmark,
                                "model": model,
                                "strategy": "Naive"
                            }
                            )

# evaluation plugin
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    loggers=[wandb_logger, InteractiveLogger()],
    strict_checks=False
)
# define strategy
cl_strategy = Naive(
    model,
    optimizer,
    criterion,
    train_mb_size=50,
    train_epochs=5,
    eval_mb_size=10,
    device=device,
    evaluator=eval_plugin
)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Evaluating on experiences until current one.')
    # test also returns a dictionary which contains all the metric values
    print(cl_strategy.eval(benchmark.test_stream[:experience.current_experience + 1]))

wandb.finish()